In [52]:
import requests
from bs4 import BeautifulSoup
import os
import csv
import re

In [53]:
stores_url = 'https://www.lenskart.com/stores'
response = requests.get(stores_url)
#extracting all the locations of lenskart stores
locations=re.findall('class="undefined".*?><.*?>(.*?)<',response.text,re.S)
locations


['Andhra Pradesh',
 'Andmaan nicobar',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chennai',
 'Chhattisgarh',
 'Dadra and Nagar Haveli',
 'Delhi',
 'Goa',
 'Gujarat',
 'Gwalior',
 'Haryana',
 'Himachal Pradesh',
 'Hyderabad',
 'Jammu and Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Nagaland',
 'New Delhi',
 'Odisha',
 'Puducherry',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Tamil\xa0Nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'Utter Pradesh',
 'Uttrakhand',
 'West Bengal']

In [54]:
for location in locations:
    url = 'https://www.lenskart.com/stores/location/' + location
    response = requests.get(url)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    stores = []
    for store in soup.find_all('div', class_='StoreCard_imgContainer__P6NMN'):
        name = store.find('a', {'class' : "StoreCard_name__mrTXJ"}).text
        address = store.find('a', {'class' : 'StoreCard_storeAddress__PfC_v'}).text
        timings = store.find('div', {'class' : 'StoreCard_storeAddress__PfC_v'}).text[7:-1]
        phone = store.find('div', {'class' : "StoreCard_wrapper__xhJ0A"}).a.text[1:]    
        ratingData = store.find('div',{'class':"StoreCard_storeRating__dJst3"})
        if ratingData is not None:
            rating = ratingData.find_all('span')[1].text
            reviewers = ratingData.find('label').text
        else:
            rating = ''
            reviewers = ''
        ratingDatas = rating  + reviewers
        #appending the datas
        stores.append([name, address, location.title(), timings,ratingDatas, phone])
    if os.path.exists('lenskart_stores.csv'):
            if os.stat('lenskart_stores.csv').st_size > 0:
                with open('lenskart_stores.csv', mode='a') as file:
                    writer = csv.writer(file,lineterminator='\n')
                    for store in stores:
                        writer.writerow(store)
    else:
        with open('lenskart_stores.csv', mode='a') as file:
            writer = csv.writer(file,lineterminator='\n')
            writer.writerow(['Store Name', 'Address', 'Location', 'Timings','Rating (no.of.reviewers)', 'Phone'])
            for store in stores:
                writer.writerow(store)